In [128]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
import matplotlib.gridspec as gridspec
from sklearn import neighbors, datasets
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_score, recall_score, f1_score

from preprocessing import *

pd.set_option('display.max_colwidth', None)
    

### 1. Datos

In [129]:
data = pd.read_csv('tp-2020-2c-train-cols2.csv')
targets = pd.read_csv('tp-2020-2c-train-cols1.csv')
df = (pd.merge(left=data, right=targets, how="left", on="id_usuario", validate="one_to_one"))
df.head()

,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede,volveria
0,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes,0
1,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes,0
2,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita,0
3,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo,0
4,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo,1


### 2. Preprocesamiento

In [130]:
#ver preprocessing.py

---

### 3. Entrenamiento

Entrenamiento con valores por defecto.

In [144]:
xtn, xts, ytn, yts = prepro_base(data, targets, train=True)

clf_knn = neighbors.KNeighborsClassifier()
resumen_modelo(clf_knn, xtn, xts, ytn, yts)

,modelo,score,precision,recall,f1,auc_roc_score
0,KNeighborsClassifier(),0.757764,0.734694,0.354839,0.396396,0.805148


---

Busqueda de hiperparámetros mediante GridSearch

In [155]:
clf_knn = neighbors.KNeighborsClassifier()

knn_grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights' : ['distance', 'uniform'],
    'metric' : ['minkowski', 'euclidean', 'manhattan']
}

gs = model_selection.GridSearchCV(
    clf_knn,
    knn_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1,
    scoring='accuracy'
)

columnas=['encoding','scaling','selection','metric', 'n_neighbors', 'weights', 'score', 'holdout_score', 'auroc_score']

aplicar_gscv2(gs, columnas, xtn, xts, ytn, yts)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


,encoding,scaling,selection,metric,n_neighbors,weights,score,holdout_score,auroc_score
0,onehot,ninguno,ninguno,manhattan,5,distance,0.784375,0.770186,0.841561


---

El mejor modelo tiene parametros ___

In [159]:
mejor_knn = neighbors.KNeighborsClassifier(metric="manhattan", n_neighbors=5, weights="distance")

resumen_modelo(mejor_knn, xtn, xts, ytn, yts)

,modelo,score,precision,recall,f1,auc_roc_score
0,"KNeighborsClassifier(metric='manhattan', weights='distance')",0.770186,0.711864,0.403226,0.413223,0.841561


### 4. Prediccion sobre los nuevos datos

In [134]:
new_data_raw = pd.read_csv('TP2-nuevos-datos.csv')
new_data = prepro(new_data_raw, None, train=False)

In [157]:
#mejor_knn.predict(new_data)

---

### 5. Codigo

In [2]:
def aplicar_gscv(gscv, columnas):
    metricas = []
    mejor_roc_auc = -1
    mejor_modelo = None
    prepro_mejor_modelo = None
    info_mejor_modelo = None
    
    for conjunto in datos: #pruebo 5 conjuntos de datos, cada uno con distintos preprocesamientos
    
        gridsearch_resultados = gscv.fit(conjunto[0], conjunto[2]) #X_train e y_train, respectivamente
        row = []
        row.append(conjunto[4]) #encoding
        row.append(conjunto[5]) #scaling
        row.append(conjunto[6]) #selection
        
        for key in gridsearch_resultados.best_params_:
            row.append(gridsearch_resultados.best_params_[key]) #Agregar el mejor valor de cada parametro testeado
        row.append(gridsearch_resultados.best_score_)           #Agregar mejor score
        row.append(gridsearch_resultados.score(conjunto[1], conjunto[3]))  #Agregar score con holdout
        
        #roc_auc
        roc_auc = roc_auc_score(conjunto[3], gridsearch_resultados.best_estimator_.predict_proba(conjunto[1])[:, 1])
        
        if(roc_auc > mejor_roc_auc):
            mejor_roc_auc = roc_auc
            mejor_modelo = gridsearch_resultados.best_estimator_
            prepro_mejor_modelo = conjunto[4] + conjunto[5] + conjunto[6]
        
        row.append(roc_auc)
        metricas.append(row)
        
    info_mejor_modelo = [mejor_modelo, prepro_mejor_modelo, mejor_roc_auc]
    mejores_modelos.append(info_mejor_modelo)
    
    resultados = pd.DataFrame(metricas, columns=columnas)
    resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
    display(resultados)

In [152]:
def aplicar_gscv2(gscv, columnas, xtn, xts, ytn, yts):
    metricas = []
    
    gridsearch_resultados = gscv.fit(xtn, ytn) #X_train e y_train, respectivamente
    row = []
    row.append("onehot") #encoding
    row.append("ninguno") #scaling
    row.append("ninguno") #selection
        
    for key in gridsearch_resultados.best_params_:
        row.append(gridsearch_resultados.best_params_[key]) #Agregar el mejor valor de cada parametro testeado
    row.append(gridsearch_resultados.best_score_)           #Agregar mejor score
    row.append(gridsearch_resultados.score(xts, yts))  #Agregar score con holdout
        
    #roc_auc
    roc_auc = roc_auc_score(yts, gridsearch_resultados.best_estimator_.predict_proba(xts)[:, 1])
    
    row.append(roc_auc)
    metricas.append(row)
    
    resultados = pd.DataFrame(metricas, columns=columnas)
    display(resultados)

In [3]:
def aplicar_gscv_nb(gscv, columnas):
    metricas = []
    mejor_roc_auc = -1
    mejor_modelo = None
    prepro_mejor_modelo = None
    
    for conjunto in datos_nb: #pruebo 5 conjuntos de datos, cada uno con distintos preprocesamientos
    
        gridsearch_resultados = gscv.fit(conjunto[0], conjunto[2]) #X_train e y_train, respectivamente
        row = []
        row.append(conjunto[4]) #encoding
        row.append(conjunto[5]) #scaling
        row.append(conjunto[6]) #selection
        
        for key in gridsearch_resultados.best_params_:
            row.append(gridsearch_resultados.best_params_[key]) #Agregar el mejor valor de cada parametro testeado
        row.append(gridsearch_resultados.best_score_)           #Agregar mejor score
        row.append(gridsearch_resultados.score(conjunto[1], conjunto[3]))  #Agregar score con holdout
        
        #roc_auc
        roc_auc = roc_auc_score(conjunto[3], gridsearch_resultados.best_estimator_.predict_proba(conjunto[1])[:, 1])
        
        #if(roc_auc > mejor_roc_auc):
        #    mejor_roc_auc = roc_auc
        #    mejor_modelo = gridsearch_resultados.best_estimator_
        
        
        row.append(roc_auc)
        
        mejores_modelos.append(gridsearch_resultados.best_estimator_)
        
        metricas.append(row)
    
    #mejores_modelos.append(mejor_modelo)
    resultados = pd.DataFrame(metricas, columns=columnas)
    resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
    display(resultados)

In [142]:
def resumen_modelo(mdl, xtn, xts, ytn, yts):
    
    mdl.fit(xtn, ytn)
    pred = mdl.predict(xts)
    mdl_score = (yts == pred)
    
    mdl_f1 = f1_score(y_test_d, mdl.predict(xts))
    mdl_recall = recall_score(y_test_d, mdl.predict(xts))
    mdl_precision = precision_score(yts, mdl.predict(xts))
    mdl_roc_auc = roc_auc_score(yts, mdl.predict_proba(xts)[:, 1])    



    metricas_mdl = pd.DataFrame([[mdl,
                                    mdl_score.mean(),
                                    mdl_precision, 
                                    mdl_recall, 
                                    mdl_f1,
                                    mdl_roc_auc]],
                                    columns=['modelo','score','precision','recall','f1','auc_roc_score'])
    display(metricas_mdl)
    